<a href="https://colab.research.google.com/github/AlineSchaub/DataScience21/blob/7_Block/7_CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment - Image Classification with CNNs
![CIFAR10](https://miro.medium.com/max/964/1*syyml8q8s1Yt-iEea5m1Ag.png)

[CIFAR 10](https://www.cs.toronto.edu/~kriz/cifar.html) is a small toy data set for image classifaction. 

* It has 60k images from ten diffenrent classes (like car, airplane, horse,..)
* The RGB images are of size 32x32 pixels  

In [1]:
#import needed libs
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

#some vars to controll the training -> use them in your code
batch_size = 32
num_classes = 10 #fixed!
epochs = 10

# Get the data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 2s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## Task:
* visualize some sample images from different classes
* build and train the ***AlexNet*** CNN architecture from the lecture, use the following Keras layers:
    * overview: https://keras.io/api/layers/
    * 2d convolution: https://keras.io/api/layers/convolution_layers/convolution2d/
    * Fully Connected (=Matrix Multiplication MLP): https://keras.io/api/layers/core_layers/dense/
    * Pooling: https://keras.io/api/layers/pooling_layers/max_pooling2d/   
* use 'relu' functions for the non-linear activation -> https://keras.io/api/layers/activation_layers/relu/
* optimize the learning hyper-paramters to get the best  test accuracy

In [2]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import CenterCrop
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import activations

#RGB (3) und Bildgröße 32 auf 32
inputs = keras.Input(shape=(32, 32, 3))


#Bilder zuschneiden
x_1 = CenterCrop(height=30, width=30)(inputs)


# Rescale images to [0, 1]
x_2 = Rescaling(scale=1.0 / 255)(x_1)

# 2d Convolution layer
x_3 = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x_2)

#fully connected layer
x_4 = layers.Dense(32, activation='relu')(x_3)

#Pooling layer
x_5 = layers.MaxPooling2D(pool_size=(3, 3))(x_4)


x_6 = layers.GlobalAveragePooling2D()(x_5)



In [3]:

outputs = layers.Dense(num_classes, activation=activations.relu)(x_6)

In [4]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
center_crop (CenterCrop)     (None, 30, 30, 3)         0         
_________________________________________________________________
rescaling (Rescaling)        (None, 30, 30, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        896       
_________________________________________________________________
dense (Dense)                (None, 28, 28, 32)        1056      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 9, 9, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0     

In [26]:
#optimize + loss
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [27]:
#batch size und epochs von oben

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True)

Epoch 1/10
1563/1563 [==============================] - 36s 22ms/step - loss: nan - accuracy: 0.1001
Epoch 2/10
1563/1563 [==============================] - 35s 22ms/step - loss: nan - accuracy: 0.0991
Epoch 3/10
1563/1563 [==============================] - 35s 22ms/step - loss: nan - accuracy: 0.1000
Epoch 4/10
1563/1563 [==============================] - 34s 22ms/step - loss: nan - accuracy: 0.0975
Epoch 5/10
1563/1563 [==============================] - 34s 22ms/step - loss: nan - accuracy: 0.0976
Epoch 6/10
1563/1563 [==============================] - 34s 22ms/step - loss: nan - accuracy: 0.1015
Epoch 7/10
1563/1563 [==============================] - 34s 22ms/step - loss: nan - accuracy: 0.0975
Epoch 8/10
1563/1563 [==============================] - 34s 22ms/step - loss: nan - accuracy: 0.1011
Epoch 9/10
1563/1563 [==============================] - 34s 22ms/step - loss: nan - accuracy: 0.1020
Epoch 10/10
1563/1563 [==============================] - 34s 22ms/step - loss: nan - accura

In [28]:
print(history.history)

{'loss': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'accuracy': [0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612]}


In [29]:
loss, acc = model.evaluate(dataset)
print("loss: %.2f" % loss)
print("acc: %.2f" % acc)

1563/1563 [==============================] - 14s 9ms/step - loss: nan - accuracy: 0.1000
loss: nan
acc: 0.10


In [31]:
import tensorflow as tf

val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

In [32]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)

In [33]:
history = model.fit(dataset, epochs=epochs, batch_size=batch_size, validation_data=val_dataset)

Epoch 1/10
1563/1563 [==============================] - 43s 28ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 2/10
1563/1563 [==============================] - 40s 26ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 3/10
1563/1563 [==============================] - 39s 25ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 4/10
1563/1563 [==============================] - 39s 25ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 5/10
1563/1563 [==============================] - 39s 25ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 6/10
1563/1563 [==============================] - 38s 25ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 7/10
1563/1563 [==============================] - 39s 25ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 8/10
1563/1563

In [34]:
loss, acc = model.evaluate(val_dataset)
print("loss: %.2f" % loss)
print("acc: %.2f" % acc)

313/313 [==============================] - 3s 9ms/step - loss: nan - accuracy: 0.1000
loss: nan
acc: 0.10


In [35]:
predictions = model.predict(val_dataset)
print(predictions)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


Hyperparameter Tuning

In [50]:
def build_model(hp):
  inputs = tf.keras.Input(shape=(32, 32, 3))
  x = inputs
  for i in range(hp.Int('conv_blocks', 3, 5, default=3)):
    filters = hp.Int('filters_' + str(i), 32, 256, step=32)
    for _ in range(2):
      x = tf.keras.layers.Convolution2D(
        filters, kernel_size=(3, 3), padding='same')(x)
      x = tf.keras.layers.BatchNormalization()(x)
      x = tf.keras.layers.ReLU()(x)
    if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max':
      x = tf.keras.layers.MaxPool2D()(x)
    else:
      x = tf.keras.layers.AvgPool2D()(x)
  x = tf.keras.layers.GlobalAvgPool2D()(x)
  x = tf.keras.layers.Dense(
      hp.Int('hidden_size', 30, 100, step=10, default=50),
      activation='relu')(x)
  x = tf.keras.layers.Dropout(
      hp.Float('dropout', 0, 0.5, step=0.1, default=0.5))(x)
  outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

  model = tf.keras.Model(inputs, outputs)
  model.compile(
    optimizer=tf.keras.optimizers.Adam(
      hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  return model

   


In [37]:
!pip install -q -U keras-tuner
from kerastuner.tuners import Hyperband


In [51]:
import kerastuner as kt

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2)

In [52]:
tuner.search_space_summary()

Search space summary
Default search space size: 10
conv_blocks (Int)
{'default': 3, 'conditions': [], 'min_value': 3, 'max_value': 5, 'step': 1, 'sampling': None}
filters_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
pooling_0 (Choice)
{'default': 'avg', 'conditions': [], 'values': ['avg', 'max'], 'ordered': False}
filters_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
pooling_1 (Choice)
{'default': 'avg', 'conditions': [], 'values': ['avg', 'max'], 'ordered': False}
filters_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
pooling_2 (Choice)
{'default': 'avg', 'conditions': [], 'values': ['avg', 'max'], 'ordered': False}
hidden_size (Int)
{'default': 50, 'conditions': [], 'min_value': 30, 'max_value': 100, 'step': 10, 'sampling': None}
dropout (Float)
{'default': 0.5, 'conditions': [], 'min_value': 0.0, 

In [55]:
best_model = tuner.get_best_models(num_models=4)


In [56]:
best_hyperparameters = tuner.get_best_hyperparameters


In [57]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
